In [1]:
import glob
from joblib import dump, load
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import warnings
import matplotlib.colors as mcolors
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr

### Create a dataframe with all campgrounds and the corresponding coordinates and nearest grid cells

In [2]:
# Define the locations and their coordinates
locations = {
    "Bern": (2595842.15, 1201548.69),
    "Brienz": (2646563.43, 1177603.88),
    "Buochs": (2674514.65, 1203568.08),
    "Interlaken": (2632995.62, 1171399.67),
    "Lugano": (2713926.15, 1094747.56),
    "Salavaux": (2569175.20, 1195846.24),
    "Sempach": (2657011.60, 1219718.50),
    "Gordevio": (2700590.96, 1119701.60),
    "Thun": (2614554.68, 1178201.33),
    "Morges": (2527116.10, 1150785.39)
}

# Open your dataset
path = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_gust_data/ICON-CH1-EPS_2024-07-11T21._agg.nc'
ds = xr.open_dataset(path)  # Replace with your actual dataset

# Loop through each location to find the nearest x and y and save variables
for name, (loc_x, loc_y) in locations.items():
    # Find the nearest grid cell
    nearest_x = ds.sel(x=loc_x, y=loc_y, method="nearest").x.values
    nearest_y = ds.sel(x=loc_x, y=loc_y, method="nearest").y.values

    # Save variables dynamically
    globals()[f"{name}_cell_x"] = nearest_x
    globals()[f"{name}_cell_y"] = nearest_y


locations_df = pd.DataFrame([(name, coord[0], coord[1]) for name, coord in locations.items()], columns=['name', 'x_coordinate', 'y_coordinate'])
# Add x_cell and y_cell columns to the dataframe
locations_df['x_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_x"))
locations_df['y_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_y"))
dump(locations_df, 'campgrounds_coordinates.pkl')

['campgrounds_coordinates.pkl']

### Load campground dataframe and select subset of dataset that only considers the chosen campground

In [3]:
campgrounds = load('campgrounds_coordinates.pkl')

In [4]:
campgrounds

,name,x_coordinate,y_coordinate,x_cell_nearest,y_cell_nearest
0,Bern,2595842.15,1201548.69,2595500.0,1201500.0
1,Brienz,2646563.43,1177603.88,2646500.0,1177500.0
2,Buochs,2674514.65,1203568.08,2674500.0,1203500.0
3,Interlaken,2632995.62,1171399.67,2632500.0,1171500.0
4,Lugano,2713926.15,1094747.56,2713500.0,1094500.0
5,Salavaux,2569175.20,1195846.24,2569500.0,1195500.0
6,Sempach,2657011.60,1219718.50,2657500.0,1219500.0
7,Gordevio,2700590.96,1119701.60,2700500.0,1119500.0
8,Thun,2614554.68,1178201.33,2614500.0,1178500.0
9,Morges,2527116.10,1150785.39,2527500.0,1150500.0


In [16]:
# Open your dataset
icon_paths = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_gust_data/ICON-CH1-EPS_2024-08-12T12._agg.nc'
ds_icon = xr.open_dataset(icon_paths)

In [17]:
# CHOOSE CAMPING SITE
c_name = 'Brienz'


# Define the window size (in grid points)
full_window = 35
half_window = int((full_window-1)/2)

center_x = campgrounds[campgrounds['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
center_y = campgrounds[campgrounds['name'].str.contains(c_name)]['y_cell_nearest'].values[0]

# Get the indices of the nearest grid point
x_index = int(ds_icon.x.to_index().get_loc(center_x))
y_index = int(ds_icon.y.to_index().get_loc(center_y))

# Compute index bounds
x_start_idx = max(x_index - half_window, 0)
x_end_idx = min(x_index + half_window + 1, ds_icon.sizes['x'])
y_start_idx = max(y_index - half_window, 0)
y_end_idx = min(y_index + half_window + 1, ds_icon.sizes['y'])

# Extract the window
window_icon = ds_icon.isel(x=slice(x_start_idx, x_end_idx), y=slice(y_start_idx, y_end_idx))

#### window_icon now contains only the data for the chosen campground with a 35kmx35km window around it

### Set the date and the leadtime as variables

In [18]:
selected_date = '2024-08-12T12'
selected_leadtime = 0

### Print statistics for a selected date, leadtime, and member

In [19]:
print(f"\n--- Some statistics for {c_name} at {selected_date}, leadtime {selected_leadtime} ---")

# Over one specific member
for member in range(11):
    print(f"\n--- Over member {member} ---")
    wind_gust_kmh = window_icon.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust * 3.6
    max_wind_gust_member = wind_gust_kmh.max().values
    min_wind_gust_member = wind_gust_kmh.min().values
    mean_wind_gust_member = wind_gust_kmh.mean().values
    

    print("Max. windspeed gust:", max_wind_gust_member, "km/h")
    print("Min. windspeed gust:", min_wind_gust_member, "km/h")
    print("Mean. windspeed gust:", mean_wind_gust_member, "km/h")
    
# Over all members
max_wind_gust = []
min_wind_gust = []
mean_wind_gust = []

for member in range(11):
    wind = window_icon.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust * 3.6
    max_wind_gust.append(wind.max().values)
    min_wind_gust.append(wind.min().values)
    mean_wind_gust.append(wind.mean().values)
    
print("\n--- Over all 11 members ---")
print("Overall max windspeed gust:", np.max(max_wind_gust), "km/h")
print("Overall min windspeed gust:", np.min(min_wind_gust), "km/h")
print("Overall mean windspeed gust:", np.mean(mean_wind_gust), "km/h")

# Save statistics to a text file
c_name_lower = c_name[0].lower() + c_name[1:]
directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
with open(f'{directory_to_save}/{selected_date}_{c_name_lower}_statistics_gust.txt', 'w') as f:
    f.write(f"\n--- Some statistics for {c_name} at {selected_date}, leadtime {selected_leadtime} ---\n")
    for member in range(11):
        wind_gust_kmh = window_icon.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust * 3.6
        max_wind_gust_member = wind_gust_kmh.max().values
        min_wind_gust_member = wind_gust_kmh.min().values
        mean_wind_gust_member = wind_gust_kmh.mean().values
        f.write(f"\n--- Over member {member} ---\n")
        f.write(f"Max. windspeed: {max_wind_gust_member} km/h\n")
        f.write(f"Min. windspeed: {min_wind_gust_member} km/h\n")
        f.write(f"Mean. windspeed: {mean_wind_gust_member} km/h\n")

    f.write(f"\n--- Over all 11 members ---\n")
    f.write(f"Overall max windspeed: {np.max(max_wind_gust)} km/h\n")
    f.write(f"Overall min windspeed: {np.min(min_wind_gust)} km/h\n")
    f.write(f"Overall mean windspeed: {np.mean(mean_wind_gust)} km/h\n")



--- Some statistics for Brienz at 2024-08-12T12, leadtime 0 ---

--- Over member 0 ---
Max. windspeed gust: 118.465996 km/h
Min. windspeed gust: 29.349234 km/h
Mean. windspeed gust: 60.811634 km/h

--- Over member 1 ---
Max. windspeed gust: 120.46746 km/h
Min. windspeed gust: 23.812445 km/h
Mean. windspeed gust: 63.259834 km/h

--- Over member 2 ---
Max. windspeed gust: 132.09607 km/h
Min. windspeed gust: 28.38117 km/h
Mean. windspeed gust: 62.97116 km/h

--- Over member 3 ---
Max. windspeed gust: 139.58086 km/h
Min. windspeed gust: 21.29529 km/h
Mean. windspeed gust: 68.784805 km/h

--- Over member 4 ---
Max. windspeed gust: 128.79105 km/h
Min. windspeed gust: 26.542673 km/h
Mean. windspeed gust: 70.65483 km/h

--- Over member 5 ---
Max. windspeed gust: 123.84183 km/h
Min. windspeed gust: 27.261608 km/h
Mean. windspeed gust: 72.20407 km/h

--- Over member 6 ---
Max. windspeed gust: 107.92064 km/h
Min. windspeed gust: 27.637196 km/h
Mean. windspeed gust: 55.966797 km/h

--- Over membe

### Plot the wind for the specific campground and the window around the campground

In [ ]:
warnings.filterwarnings("ignore")

# plot the actual data, for a specific date, time step, and member
for member_number in range(11): # Loop through members 0 to 10
    data_wind = window_icon.sel(date=selected_date, time=selected_leadtime, members=member_number).wind_speed

    # Define the Swiss coordinate system (EPSG:2056)
    swiss_proj = ccrs.epsg(2056)

    # Compute the extent of the data
    min_x, max_x = data_wind.x.values.min(), data_wind.x.values.max()
    min_y, max_y = data_wind.y.values.min(), data_wind.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    # Dynamically adjust figure size to maintain a square aspect ratio
    fig_size = 13  # Base figure size
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    # Create the figure and axes
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')  # Ensure square aspect ratio for the map

    # Set the extent of the map
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)


    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders


    # Define colormap and normalization
    colors = [
    '#cccccc',  # 0–12
    '#66ff00',  # 12–24
    "#b3ff00",  # 24–36
    "#fffb00",  # 36–48
    '#ff9900',  # 48–60
    "#ff3c00",  # 60–72
    '#ff0000',  # 72–84
    "#ff0062",  # 84–96
    "#ff00bfdd",  # 96–108
    "#ae00ff3e",  # >108
    ]
    cmap = mcolors.ListedColormap(colors)
    bounds = [0, 12, 24, 36, 48, 60, 72, 84, 96, 108]
    norm = mcolors.BoundaryNorm(bounds, cmap.N - 1)


    # Convert wind speed to km/h for plotting
    plot_data = data_wind * 3.6

    # Plot the data (use plot_data instead of data)
    im = ax.contourf(
        plot_data.x,
        plot_data.y,
        plot_data,
        levels=bounds,
        vmin=1 * 3.6, vmax=60 * 3.6,
        transform=swiss_proj,
        cmap=cmap, norm=norm,
        alpha=0.85, extend='max'
    )


    # Add the colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Wind Speed [km/h]', fontsize=14)
    cbar.ax.tick_params(labelsize=14)

    # Add dot where camping site is located
    ax.plot(campgrounds[campgrounds['name'].str.contains(c_name)]['x_cell_nearest'].values[0], campgrounds[campgrounds['name'].str.contains(c_name)]['y_cell_nearest'].values[0], 'ro', transform=swiss_proj)

    # Extract date, time, and member values
    date_val = str(data_wind.coords['date'].values)
    time_val = int(data_wind.coords['time'].values)
    member_val = int(data_wind.coords['members'].values)


    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_windspeed_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')
    


### Plot the precipitation for the specific campground and the window around the campground

In [ ]:
warnings.filterwarnings("ignore")

# Plot the actual data, for a specific date, time step, and member
for member_number in range(11): # Loop through members 0 to 10
    data_prec = window_icon.sel(date=selected_date, time=selected_leadtime, members=member_number).tot_prec_1h

    # Define the Swiss coordinate system (EPSG:2056)
    swiss_proj = ccrs.epsg(2056)

    # Compute the extent of the data
    min_x, max_x = data_prec.x.values.min(), data_prec.x.values.max()
    min_y, max_y = data_prec.y.values.min(), data_prec.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    # Dynamically adjust figure size to maintain a square aspect ratio
    fig_size = 13  # Base figure size
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    # Create the figure and axes
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')  # Ensure square aspect ratio for the map

    # Set the extent of the map
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)


    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders


    # Define colormap and normalization
    colors = ['#eafac3', '#b3f8c0', '#58c1a0', '#32a1a0', '#0f79b1', '#06488e', '#1f0d64', '#6a2c5a']
    cmap = mcolors.ListedColormap(colors)
    bounds = [1, 2, 4, 6, 10, 20, 40, 60]
    norm = mcolors.BoundaryNorm(bounds, cmap.N - 1)


    # Plot the data
    im = ax.contourf(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=bounds,
        vmin=1, vmax=60,
        transform=swiss_proj,
        cmap=cmap, norm=norm,
        alpha=0.85, extend='max'
    )


    # Add the colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30,shrink=0.825)
    cbar.set_label('Precipitation [mm/h]', fontsize=14)
    cbar.ax.tick_params(labelsize=14)

    # Add dot where camping site is located
    ax.plot(campgrounds[campgrounds['name'].str.contains(c_name)]['x_cell_nearest'].values[0], campgrounds[campgrounds['name'].str.contains(c_name)]['y_cell_nearest'].values[0], 'ro', transform=swiss_proj)

    # Extract date, time, and member values
    date_val = str(data_prec.coords['date'].values)
    time_val = int(data_prec.coords['time'].values)
    member_val = int(data_prec.coords['members'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]

    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_precipitation_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')



### Place the wind contours over the precipitation plot for the specific campground and the window around the campground

In [ ]:
from matplotlib.lines import Line2D

warnings.filterwarnings("ignore")

for member_number in range(11):  # Loop through members 0 to 10
    data_prec = window_icon.sel(date=selected_date, time=selected_leadtime, members=member_number).tot_prec_1h
    data_wind = window_icon.sel(date=selected_date, time=selected_leadtime, members=member_number).wind_speed * 3.6  # Convert to km/h

    swiss_proj = ccrs.epsg(2056)

    min_x, max_x = data_prec.x.values.min(), data_prec.x.values.max()
    min_y, max_y = data_prec.y.values.min(), data_prec.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    fig_size = 13
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders

    prec_colors = ['#eafac3', '#b3f8c0', '#58c1a0', '#32a1a0', '#0f79b1', '#06488e', '#1f0d64', '#6a2c5a']
    prec_cmap = mcolors.ListedColormap(prec_colors)
    prec_bounds = [1, 2, 4, 6, 10, 20, 40, 60]
    prec_norm = mcolors.BoundaryNorm(prec_bounds, prec_cmap.N - 1)

    im = ax.contourf(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=prec_bounds,
        vmin=1, vmax=60,
        transform=swiss_proj,
        cmap=prec_cmap, norm=prec_norm,
        alpha=0.85, extend='max'
    )

    # Wind contour thresholds and colors
    bounds = [12, 24, 36, 48, 60, 72, 84, 96, 108]
    wind_colors = [
        '#cccccc',  # 0–12
        '#66ff00',  # 12–24
        "#b3ff00",  # 24–36
        "#fffb00",  # 36–48
        '#ff9900',  # 48–60
        "#ff3c00",  # 60–72
        '#ff0000',  # 72–84
        "#ff0062",  # 84–96
        "#ff00bfdd",  # 96–108
        "#ae00ff3e",  # >108
    ][:len(bounds)]

    wind_contour_levels = bounds

    wind_contour = ax.contour(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=wind_contour_levels,
        colors=wind_colors,
        linewidths=2.2,
        transform=swiss_proj
    )

    ax.clabel(wind_contour, inline=True, fontsize=18, fmt='%d')

    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Precipitation [mm/h]', fontsize=14)
    cbar.ax.tick_params(labelsize=14)

    # Add legend for wind speed contours
    wind_legend_lines = [Line2D([0], [0], color=color, lw=2.2) for color in wind_colors]
    wind_legend_labels = [f'{b} km/h' for b in bounds]
    legend = ax.legend(
        handles=wind_legend_lines,
        labels=wind_legend_labels,
        loc='upper left',
        bbox_to_anchor=(-0.235, 1.0),
        fontsize=14,
        framealpha=0.9,
        title='Wind Speed\nContours [km/h]',
        title_fontsize=14
    )
    legend._legend_box.align = "left"

    ax.plot(
        campgrounds[campgrounds['name'].str.contains(c_name)]['x_cell_nearest'].values[0],
        campgrounds[campgrounds['name'].str.contains(c_name)]['y_cell_nearest'].values[0],
        'ro', transform=swiss_proj
    )

    date_val = str(data_prec.coords['date'].values)
    time_val = int(data_prec.coords['time'].values)
    member_val = int(data_prec.coords['members'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]

    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_precipitation_windcontours_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')


### Wind contours plotted over precipitation only for 48 km/h wind speed and above

In [ ]:
from matplotlib.lines import Line2D

warnings.filterwarnings("ignore")

for member_number in range(11):  # Loop through members 0 to 10
    data_prec = window_icon.sel(date=selected_date, time=selected_leadtime, members=member_number).tot_prec_1h
    data_wind = window_icon.sel(date=selected_date, time=selected_leadtime, members=member_number).wind_speed * 3.6  # Convert to km/h

    swiss_proj = ccrs.epsg(2056)

    min_x, max_x = data_prec.x.values.min(), data_prec.x.values.max()
    min_y, max_y = data_prec.y.values.min(), data_prec.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    fig_size = 13
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders

    prec_colors = ['#eafac3', '#b3f8c0', '#58c1a0', '#32a1a0', '#0f79b1', '#06488e', '#1f0d64', '#6a2c5a']
    prec_cmap = mcolors.ListedColormap(prec_colors)
    prec_bounds = [1, 2, 4, 6, 10, 20, 40, 60]
    prec_norm = mcolors.BoundaryNorm(prec_bounds, prec_cmap.N - 1)

    im = ax.contourf(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=prec_bounds,
        vmin=1, vmax=60,
        transform=swiss_proj,
        cmap=prec_cmap, norm=prec_norm,
        alpha=0.85, extend='max'
    )

    # Wind contour thresholds and colors
    bounds = [12, 24, 36, 48, 60, 72, 84, 96, 108]
    wind_colors = [
        '#cccccc',  # 0–12
        '#66ff00',  # 12–24
        "#b3ff00",  # 24–36
        "#fffb00",  # 36–48
        '#ff9900',  # 48–60
        "#ff3c00",  # 60–72
        '#ff0000',  # 72–84
        "#ff0062",  # 84–96
        "#ff00bfdd",  # 96–108
        "#ae00ff3e",  # >108
    ][:len(bounds)]

    # Only show contours above 48 km/h
    wind_contour_levels = [b for b in bounds if b >= 48]

    wind_contour = ax.contour(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=wind_contour_levels,
        colors=wind_colors[-len(wind_contour_levels):],
        linewidths=2.2,
        transform=swiss_proj
    )

    ax.clabel(wind_contour, inline=True, fontsize=18, fmt='%d')

    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Precipitation [mm/h]', fontsize=14)
    cbar.ax.tick_params(labelsize=14)

    # Add legend for wind speed contours
    wind_legend_lines = [Line2D([0], [0], color=color, lw=2.2) for color in wind_colors]
    wind_legend_labels = [f'{b} km/h' for b in bounds]
    legend = ax.legend(
        handles=wind_legend_lines,
        labels=wind_legend_labels,
        loc='upper left',
        bbox_to_anchor=(-0.235, 1.0),
        fontsize=14,
        framealpha=0.9,
        title='Wind Speed\nContours [km/h]',
        title_fontsize=14
    )
    legend._legend_box.align = "left"

    ax.plot(
        campgrounds[campgrounds['name'].str.contains(c_name)]['x_cell_nearest'].values[0],
        campgrounds[campgrounds['name'].str.contains(c_name)]['y_cell_nearest'].values[0],
        'ro', transform=swiss_proj
    )

    date_val = str(data_prec.coords['date'].values)
    time_val = int(data_prec.coords['time'].values)
    member_val = int(data_prec.coords['members'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]

    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_precipitation_windcontours_above48_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')


### Place the wind contours over the precipitation plot for the specific campground and the window around the campground
#### All 11 members on one plot

### Wind contours plotted over precipitation only for 48 km/h wind speed and above 
#### All 11 members on one plot

### Create a scatterplot for a first visualization of possible correlations between wind and precipitation

### Print out the important statistical numbers